# Manejo de datos duplicados, faltantes o inválidos

## Conjunto de datos a utilizar
El objetivo notebook es realizar una inspección preliminar del dataset que fue creado en la práctica 1 por los mismos alumnos del máster. El dataset se denomina Valores Climatológicos Diarios de España, VCDE. Para esta práctica nos hemos quedado con los valores climatológicos diarios de España procedentes del sistema AEMET OpenData. [AEMET OpenData](https://opendata.aemet.es)

## Cargado de datos
Cargaremos los datos procedentes de `data/VCDE.csv` para que podamos leerlo.

## Configuración
Necesitamos importar`pandas` y `numpy` y empezar a leer el conjunto de datos

In [46]:
import pandas as pd
import numpy as np


df = pd.read_csv(
    'data/VCDE.csv', 
    usecols=['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec','tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha','horaracha', 'sol', 'presmax', 'horapresmax', 'presmin', 'horapresmin']
)

## Búsqueda de los datos problemáticos
Empezamos mirando las primeras líneas

In [47]:
df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presmax,horapresmax,presmin,horapresmin
0,2021-01-01,0252D,ARENYS DE MAR,BARCELONA,74,"7,6","0,0","4,6",Varias,"10,6",13:00,NE,"0,8","6,9",08:10,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,0252D,ARENYS DE MAR,BARCELONA,74,"7,3","0,0","3,5",05:40,"11,1",11:40,NE,"1,1","4,7",22:30,NaN,NaN,NaN,NaN,NaN
2,2021-01-03,0252D,ARENYS DE MAR,BARCELONA,74,"6,3","0,0","2,0",08:20,"10,6",12:40,NNE,"1,9","5,0",20:10,NaN,NaN,NaN,NaN,NaN
3,2021-01-04,0252D,ARENYS DE MAR,BARCELONA,74,"6,4","0,0","1,5",04:00,"11,2",11:20,NNE,"1,1","7,2",06:10,NaN,NaN,NaN,NaN,NaN
4,2021-01-05,0252D,ARENYS DE MAR,BARCELONA,74,"6,6","0,3","1,7",23:20,"11,4",12:20,NNE,"1,4","5,0",11:10,NaN,NaN,NaN,NaN,NaN


La observación de las estadísticas resumidas puede revelar valores extraños o ausentes:

In [48]:
df.describe()

,altitud,presmax,horapresmax,presmin,horapresmin
count,3.090418e+06,0.0,0.0,0.0,0.0
mean,4.323069e+02,NaN,NaN,NaN,NaN
std,4.370447e+02,NaN,NaN,NaN,NaN
min,1.000000e+00,NaN,NaN,NaN,NaN
25%,4.800000e+01,NaN,NaN,NaN,NaN
50%,3.360000e+02,NaN,NaN,NaN,NaN
75%,6.900000e+02,NaN,NaN,NaN,NaN
max,2.371000e+03,NaN,NaN,NaN,NaN


El método `info()` puede señalar los valores que faltan y los tipos de datos erróneos:

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3090418 entries, 0 to 3090417
Data columns (total 20 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fecha        object 
 1   indicativo   object 
 2   nombre       object 
 3   provincia    object 
 4   altitud      int64  
 5   tmed         object 
 6   prec         object 
 7   tmin         object 
 8   horatmin     object 
 9   tmax         object 
 10  horatmax     object 
 11  dir          object 
 12  velmedia     object 
 13  racha        object 
 14  horaracha    object 
 15  sol          object 
 16  presmax      float64
 17  horapresmax  float64
 18  presmin      float64
 19  horapresmin  float64
dtypes: float64(4), int64(1), object(15)
memory usage: 471.6+ MB


## Comprobación de nulos

Podemos utilizar el método `isna()`/`isnull()` de la serie para encontrar nulos. En nuestro ejemplo, todas las filas tiene algún campo nulo. Por lo que debemos tratar este punto con detenimiento.

In [50]:
contain_nulls = df[
    df.fecha.isna() | df.indicativo.isna() | df.nombre.isna()
    | df.horatmin.isna() | df.altitud.isna() | df.tmed.isna() | df.prec.isna() |  df.tmin.isna() |  df.horatmin.isna() 
    | df.tmax.isna() |  df.horatmax.isna() | df.dir.isna() | df.velmedia.isna() | df.racha.isna()
    |df.horaracha.isna() | df.sol.isna() |  df.presmax.isna() |  df.presmin.isna() |  df.horapresmin.isna()
]

contain_nulls.shape[0]

3090418



Obtenemos muchos registros con valores nulos, por lo tanto, hay que hacer un tratamiento de datos.

## Identificación de las columnas con valores NaN
Podemos encontrar los valores NaN de cada columna, por ejemplo, para la columna prec, se tiene:

In [51]:
df[df.prec.isna()].shape[0]

103344

Para evitar tener que ir columna por columna, podemos escribir una función que utilice un [dictionary comprehension](https://www.python.org/dev/peps/pep-0274/) que nos chequee todos los valores:

In [52]:
def get_inf_nulos(df):
    """Encontrar el número de valores nulos o nan por columna en el marco de datos"""
    return {
        col: df[df[col].isna()].shape[0] for col in df.columns
    }

get_inf_nulos(df)

{'fecha': 0,
 'indicativo': 0,
 'nombre': 0,
 'provincia': 0,
 'altitud': 0,
 'tmed': 139882,
 'prec': 103344,
 'tmin': 137564,
 'horatmin': 284767,
 'tmax': 137969,
 'horatmax': 282309,
 'dir': 481795,
 'velmedia': 302349,
 'racha': 481960,
 'horaracha': 484077,
 'sol': 1315853,
 'presmax': 3090418,
 'horapresmax': 3090418,
 'presmin': 3090418,
 'horapresmin': 3090418}


Por lo tanto, nuestro conjunto de datos sí que tiene valores nulos o ausentes en las siguientes columnas:

'tmed'
'prec'
'tmin'
'horatmin'
'tmax'
'horatmax'
'dir'
'velmedia'
'racha'
'horaracha'


Se observa que las columnas 'presmax', 'horapresmax', 'presmin' y 'horapresmin' contiene tantos valores ausentes como número de registros tiene el dataset. Por lo tanto, estas columnas serán eliminadas.

Por otra parte, la columna sol también tiene muchos valores ausentes y además el estudio no se va a realizar sobre el parámetro insolación, por lo tanto, también eliminamos la columna sol.

Tampoco trabajaremos con horatmin, horatmax ni horaracha por lo poco que aportan los valores de hora respecto del estudio que queremos realizar.


In [53]:
del df['presmax']
del df['horapresmax']
del df['presmin']
del df['horapresmin']
del df['sol']
del df['horatmin']
del df['horatmax']
del df['horaracha']

df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,dir,velmedia,racha
0,2021-01-01,0252D,ARENYS DE MAR,BARCELONA,74,"7,6","0,0","4,6","10,6",NE,"0,8","6,9"
1,2021-01-02,0252D,ARENYS DE MAR,BARCELONA,74,"7,3","0,0","3,5","11,1",NE,"1,1","4,7"
2,2021-01-03,0252D,ARENYS DE MAR,BARCELONA,74,"6,3","0,0","2,0","10,6",NNE,"1,9","5,0"
3,2021-01-04,0252D,ARENYS DE MAR,BARCELONA,74,"6,4","0,0","1,5","11,2",NNE,"1,1","7,2"
4,2021-01-05,0252D,ARENYS DE MAR,BARCELONA,74,"6,6","0,3","1,7","11,4",NNE,"1,4","5,0"


## Valores infinitos o muy grandes
Podemos escribir una función que utilice un [dictionary comprehension](https://www.python.org/dev/peps/pep-0274/) que nos chequee todas las columnas en búsqueda de valores infinitos o muy grandes.

In [55]:
def get_inf_count(df):
    """Encontrar el número de valores inf/inf por columna en el marco de datos"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }

get_inf_count(df)

{'fecha': 0,
 'indicativo': 0,
 'nombre': 0,
 'provincia': 0,
 'altitud': 0,
 'tmed': 0,
 'prec': 0,
 'tmin': 0,
 'tmax': 0,
 'dir': 0,
 'velmedia': 0,
 'racha': 0}

Se comprueba fácilmente que nuestro conjunto de datos no tiene valores infinitos.

### Tratamiento de valores null
Los valores nulos de las columnas que anteriormente hemos detectado debemos tratarlos siguiendo diferentes técnicas dependiendo de la naturaleza del dato. 

Entre las diferentes opciones existentes residen:

a) eliminar los nulos (esta opción no nos interesa pues perderíamos muchos registros del dataset)

b) sustituirlos por algún valor arbitrario

c) imputarlos utilizando los datos circundantes

Cada una de estas opciones puede tener consecuencias, por lo que debemos elegir sabiamente.Analizaremos a continuación cada una de las variables.


### Caso: tmed

Se trata de la temperatura media. En este caso vamos a completar mediante el empleo del método `fillna()`

Hacemos la suposición de que la temperatura media no cambiará drásticamente día a día. Nótese que esto es realmente una gran suposición, pero nos permitirá entender cómo funciona `fillna()` cuando proporcionamos una estrategia a través del parámetro `method`. El método `fillna()` nos da 2 opciones para el parámetro `method`:
- `'ffill'` para rellenar hacia adelante
- `'bfill'` para rellenar hacia atrás


In [56]:

df=df.assign(
    tmed=lambda x: x.tmed.fillna(method='ffill'),
)
df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,dir,velmedia,racha
0,2021-01-01,0252D,ARENYS DE MAR,BARCELONA,74,"7,6","0,0","4,6","10,6",NE,"0,8","6,9"
1,2021-01-02,0252D,ARENYS DE MAR,BARCELONA,74,"7,3","0,0","3,5","11,1",NE,"1,1","4,7"
2,2021-01-03,0252D,ARENYS DE MAR,BARCELONA,74,"6,3","0,0","2,0","10,6",NNE,"1,9","5,0"
3,2021-01-04,0252D,ARENYS DE MAR,BARCELONA,74,"6,4","0,0","1,5","11,2",NNE,"1,1","7,2"
4,2021-01-05,0252D,ARENYS DE MAR,BARCELONA,74,"6,6","0,3","1,7","11,4",NNE,"1,4","5,0"


### Caso: prec, velmedia, racha

En este caso lo mejor es asignar un valor cero en ausencia de precipitación. 

Podemos utilizar `np.nan_to_num()` para convertir `np.nan` en 0 y `-np.inf`/`np.inf` en grandes números finitos negativos o positivos. Para ello, aplicamos el siguiente código:

In [57]:
df = df.assign(
    prec=lambda x: np.nan_to_num(x.prec),
    velmedia=lambda x: np.nan_to_num(x.velmedia),
    racha=lambda x: np.nan_to_num(x.racha),
)
df.head()



,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,dir,velmedia,racha
0,2021-01-01,0252D,ARENYS DE MAR,BARCELONA,74,"7,6","0,0","4,6","10,6",NE,"0,8","6,9"
1,2021-01-02,0252D,ARENYS DE MAR,BARCELONA,74,"7,3","0,0","3,5","11,1",NE,"1,1","4,7"
2,2021-01-03,0252D,ARENYS DE MAR,BARCELONA,74,"6,3","0,0","2,0","10,6",NNE,"1,9","5,0"
3,2021-01-04,0252D,ARENYS DE MAR,BARCELONA,74,"6,4","0,0","1,5","11,2",NNE,"1,1","7,2"
4,2021-01-05,0252D,ARENYS DE MAR,BARCELONA,74,"6,6","0,3","1,7","11,4",NNE,"1,4","5,0"


Además para el caso de la precipitación reemplazamos Ip, Acum (valores inferiores a 1 mm por 0,0)

In [62]:
df['prec'] = df['prec'].str.replace('Ip','0,0')
df['prec'] = df['prec'].str.replace('Acum','0,0')
df.head()


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,dir,velmedia,racha
0,2021-01-01,0252D,ARENYS DE MAR,BARCELONA,74,"7,6","0,0","4,6","10,6",NE,"0,8","6,9"
1,2021-01-02,0252D,ARENYS DE MAR,BARCELONA,74,"7,3","0,0","3,5","11,1",NE,"1,1","4,7"
2,2021-01-03,0252D,ARENYS DE MAR,BARCELONA,74,"6,3","0,0","2,0","10,6",NNE,"1,9","5,0"
3,2021-01-04,0252D,ARENYS DE MAR,BARCELONA,74,"6,4","0,0","1,5","11,2",NNE,"1,1","7,2"
4,2021-01-05,0252D,ARENYS DE MAR,BARCELONA,74,"6,6","0,3","1,7","11,4",NNE,"1,4","5,0"


Guardamos el dataset

In [59]:
df.to_csv('data/VCDE_002.csv', sep='|')